In [1]:
print('Hello')

Hello


In [19]:
#Updated 03 Oct 2023
import win32com.client
# import Common
from typing import List, Dict, Any
import os
from time import sleep

ComServer = win32com.client.Dispatch("SQLAcc.BizApp")

def KillApp():
    os.system('cmd /c "taskkill /IM "SQLACC.exe" /F"')
    sleep(2) #sleep 2 sec


def CheckLogin():
    global ComServer
    ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
    B = ComServer.IsLogin
    if B == True:
        ComServer.Logout()
        KillApp()
    ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
    try:    
        ComServer.Login("ADMIN", "ADMIN", #UserName, Password
                        "\\DESKTOP-C8CJ70C\eStream\SQLAccounting\Share\Default.DCF", #DCF file
                        "ACC-0001.FDB") #Database Name
    except Exception as e:
        print("Oops !", e)


def ShowResult(ADataset):
    if ADataset.RecordCount > 0:
        while not ADataset.eof:
            fc = ADataset.Fields.Count
            for x in range(fc):
                fn = ADataset.Fields.Items(x).FieldName
                fv = ADataset.FindField(fn).AsString
                lresult = "Index : "+ str(x) + " FieldName : " + fn + " Value : " + fv
                print (lresult)
            print("====")
            ADataset.Next()
    else:
        print ("Record Not Found")
        
        
def GetResult(ADataset) -> List[Dict[str, Any]] | None:
    Result = []
    if ADataset.RecordCount > 0:
        while not ADataset.eof:
            fc = ADataset.Fields.Count
            lData = {}
            for x in range(fc):
                fn = ADataset.Fields.Items(x).FieldName
                fv = ADataset.FindField(fn).AsString
                lData[fn] = fv
            Result.append(lData)
            ADataset.Next()
        return Result
    else:
        return None
        
def QuotedStr(ACode):
    return "'" + ACode.replace("'", "''") + "'"
	
# login
# CheckLogin()

    

def GetListData():
    lSQL = "SELECT * FROM AGENT "
    lDataSet = ComServer.DBManager.NewDataSet(lSQL)
    return GetResult(lDataSet)



GetListData()

<>:25: SyntaxWarning: invalid escape sequence '\e'
<>:25: SyntaxWarning: invalid escape sequence '\e'
C:\Users\D&B Trading\AppData\Local\Temp\ipykernel_940\1505588290.py:25: SyntaxWarning: invalid escape sequence '\e'
  "\\DESKTOP-C8CJ70C\eStream\SQLAccounting\Share\Default.DCF", #DCF file


[{'CODE': '----',
  'DESCRIPTION': 'DEFAULT AGENT',
  'ATTACHMENTS': '',
  'ISACTIVE': 'True'}]

## Example config toml

```toml
[SQLACC]
username = "USER"
password = "PASSWORD"
dfc = "\\\\path\\Default.DCF"
fdb = "DATABASE.FDB"
```

In [37]:
import win32com.client
from typing import List, Dict, Any
import os
from time import sleep
import tomllib

with open('config.toml', 'rb') as f:
    sqlacc = tomllib.load(f)['SQLACC']
    
ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
class ServerManager:
    @classmethod
    def KillApp(cls):
        os.system('cmd /c "taskkill /IM "SQLACC.exe" /F"')
        sleep(2) #sleep 2 sec
    @classmethod
    def CheckLogin(cls):
        global ComServer
        ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
        B = ComServer.IsLogin
        if B == True:
            ComServer.Logout()
            cls.KillApp()
        ComServer = win32com.client.Dispatch("SQLAcc.BizApp")
        try:    
            ComServer.Login(sqlacc.get('username', ''), sqlacc.get('password', ''), #UserName, Password
                            sqlacc.get('dfc', ''), #DCF file
                            sqlacc.get('fdb', '')) #Database Name
        except Exception as e:
            print("Oops !", e)
            
    @classmethod
    def Get():
        return ComServer
            
class SQLUtils:
    @classmethod
    def ShowResult(cls, ADataset):
        if ADataset.RecordCount > 0:
            while not ADataset.eof:
                fc = ADataset.Fields.Count
                for x in range(fc):
                    fn = ADataset.Fields.Items(x).FieldName
                    fv = ADataset.FindField(fn).AsString
                    lresult = "Index : "+ str(x) + " FieldName : " + fn + " Value : " + fv
                    print (lresult)
                print("====")
                ADataset.Next()
        else:
            print ("Record Not Found")
            
    @classmethod
    def GetResult(cls, ADataset) -> List[Dict[str, Any]] | None:
        Result = []
        if ADataset.RecordCount > 0:
            while not ADataset.eof:
                fc = ADataset.Fields.Count
                lData = {}
                for x in range(fc):
                    fn = ADataset.Fields.Items(x).FieldName
                    fv = ADataset.FindField(fn).AsString
                    lData[fn] = fv
                Result.append(lData)
                ADataset.Next()
            return Result
        else:
            return None


# lSQL = "SELECT * FROM AR_CT A "
# lSQL += "INNER JOIN AR_KNOCKOFF B ON (A.DOCKEY = B.DOCKEY)"
lSQL = """SELECT * FROM AR_CT A 
INNER JOIN AR_KNOCKOFF B ON (A.DOCKEY = B.DOCKEY)
;
"""
lDataSet = ComServer.DBManager.NewDataSet(lSQL)
res = SQLUtils.GetResult(lDataSet)
import pprint
pprint.pprint(res)

# BizObject = ComServer.BizObjects.Find('AR_CT')
# lDetailDataSet = BizObject.DataSets.Find('cdsKnockOff')
# # Select(Fields, Where, OrderBy, OutputFormat, Delimiter, OutputFile): String; 
# FileName = lDetailDataSet.Select("*", "", "", "AD", ",", "C:\\Users\\D&B Trading\\Downloads\\AR_CT.json")

[{'ACTUALLOCALKOAMT': '20',
  'AGENT': '----',
  'AREA': '----',
  'ATTACHMENTS': '',
  'CANCELLED': 'False',
  'CODE': '300-1026',
  'CURRENCYCODE': '----',
  'CURRENCYRATE': '1',
  'DESCRIPTION': 'Contra',
  'DOCAMT': '20',
  'DOCDATE': '31/01/2025',
  'DOCKEY': '1',
  'DOCKEY_1': '1',
  'DOCNO': 'CT-00001',
  'FROMDOCKEY': '1',
  'FROMDOCTYPE': 'CT',
  'GAINLOSS': '0',
  'GAINLOSSPOSTDATE': '',
  'GLTRANSID': '481',
  'JOURNAL': 'GENERAL',
  'KOAMT': '20',
  'KOTAXDATE': '31/01/2025',
  'LASTMODIFIED': '1742173111',
  'LOCALDOCAMT': '20',
  'LOCALKOAMT': '20',
  'NOTE': '',
  'POSTDATE': '31/01/2025',
  'PROJECT': '----',
  'TAXDATE': '31/01/2025',
  'TODOCKEY': '175',
  'TODOCTYPE': 'IV',
  'UNAPPLIEDAMT': '0',
  'UPDATECOUNT': '1'},
 {'ACTUALLOCALKOAMT': '50',
  'AGENT': '----',
  'AREA': '----',
  'ATTACHMENTS': '',
  'CANCELLED': 'False',
  'CODE': '300-1034',
  'CURRENCYCODE': '----',
  'CURRENCYRATE': '1',
  'DESCRIPTION': 'Contra',
  'DOCAMT': '50',
  'DOCDATE': '31/01/2025',

In [39]:


class SQLObject:
    ...
    def select(self):
      ...
      return f"""SELECT {" ".join(self.fields)} FROM {self.tablename}"""
    
class AR_CT(SQLObject):   
    ...
    tablename = 'AR_CT'
    fields = [
        'DOCKEY', 'DOCNO', 'CODE', 'DOCDATE', 'POSTDATE', 'TAXDATE', 'DESCRIPTION', 'AREA', 'AGENT', 'JOURNAL', 'PROJECT', 'CURRENCYCODE', 'CURRENCYRATE', 'DOCAMT', 'LOCALDOCAMT', 'UNAPPLIEDAMT', 'GLTRANSID', 'CANCELLED', 'UPDATECOUNT', 'ATTACHMENTS', 'NOTE', 'LASTMODIFIED', 'DOCNOSETKEY', 'NEXTDOCNO', 'CHANGED', 'sdsKnockOff'
    ]
    
ar_obj = AR_CT()
print(ar_obj.select())

SELECT DOCKEY DOCNO CODE DOCDATE POSTDATE TAXDATE DESCRIPTION AREA AGENT JOURNAL PROJECT CURRENCYCODE CURRENCYRATE DOCAMT LOCALDOCAMT UNAPPLIEDAMT GLTRANSID CANCELLED UPDATECOUNT ATTACHMENTS NOTE LASTMODIFIED DOCNOSETKEY NEXTDOCNO CHANGED sdsKnockOff FROM AR_CT
